# Model training and evaluation

Link to datatset: [open here](https://bark.phon.ioc.ee/voxlingua107/)

Python 3.12.7 recommended

Folder structure:
```
├── data
│   ├── en
│   │   ├── en_1.wav
│   │   ├── en_2.wav
│   │   ├── ...
│   ├── fr
│   │   ├── fr_1.wav
│   │   ├── fr_2.wav
│   │   ├── ...
│   ├── ...
language.ipynb
```

In [ ]:
# DOWNLOAD VOXLINGUA107 DATASET WITH THIS SCRIPT

import os
import requests
from tqdm import tqdm
from multiprocessing import Pool
from zipfile import ZipFile

# Base URL for VoxLingua107 zip files
base_url = "https://bark.phon.ioc.ee/voxlingua107/"

# Directory to store the downloaded data
data_dir = "data"

# Function to download and extract a language zip file
def download_and_extract(url):
    lang_code = url.split("/")[-1].split(".")[0]
    lang_dir = os.path.join(data_dir, lang_code)

    # Create directory for the language if it doesn't exist
    if not os.path.exists(lang_dir):
        os.makedirs(lang_dir)

    # Path to save the zip file
    zip_path = os.path.join(lang_dir, f"{lang_code}.zip")

    # Download the zip file
    if not os.path.exists(zip_path):
        response = requests.get(url, stream=True)
        total_size = int(response.headers.get('content-length', 0))
        with open(zip_path, 'wb') as f:
            for data in tqdm(response.iter_content(1024), total=total_size // 1024, unit='KB'):
                f.write(data)

    # Extract the zip file
    with ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(lang_dir)

    # Remove the zip file after extraction
    os.remove(zip_path)

# Function to download the language data for the specified languages
def download_languages(languages):
    # Get the URLs of all language zip files
    url_list = requests.get(base_url + "zip_urls.txt").text.splitlines()

    # Filter the URLs for the specified languages
    selected_urls = [url for url in url_list if any(lang in url for lang in languages)]

    # Download and extract the selected languages in parallel
    with Pool(4) as p:
        p.map(download_and_extract, selected_urls)

if __name__ == "__main__":
    # Example: list of language codes to download (e.g., "en" for English, "fr" for French)
    languages_to_download = ["en", "fr", "de", "es", "it", "pt", "ru", "tr"]  # Add or modify the languages here

    download_languages(languages_to_download)


In [1]:
# Install libraries
# %pip install tensorflow, numpy, librosa, numpy

%load_ext tensorboard

In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D, Resizing
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.image import resize
from tensorflow.keras.models import load_model
import tqdm
import datetime
import matplotlib.pyplot as plt

In [3]:
# Varibles for the model
batch_size = 32
epochs = 80
target_shape = (512, 512)
max_files_per_class = 4000

In [ ]:
# Function to convert waveform to spectrogram
def get_spectrogram(waveform):
    # Convert the waveform to a spectrogram using Short-Time Fourier Transform (STFT)
    spectrogram = tf.signal.stft(waveform, frame_length=255, frame_step=128)
    # Obtain the magnitude of the STFT
    spectrogram = tf.abs(spectrogram)
    # Add a `channels` dimension so that the spectrogram can be used as image-like input data
    spectrogram = spectrogram[..., tf.newaxis]
    return spectrogram

# Function to load and preprocess data using TensorFlow
def load_and_preprocess_data_tf(data_dir, classes, target_shape=(128, 128), max_files_per_class=100):
    print('Loading and preprocessing data with TensorFlow...')
    data = []
    labels = []

    for i, class_name in enumerate(classes):
        print(f'Processing class {class_name} ({i+1}/{len(classes)})...')
        class_dir = os.path.join(data_dir, class_name)
        file_count = 0
        for filename in tqdm.tqdm(os.listdir(class_dir)):
            if filename.endswith('.wav'):
                file_path = os.path.join(class_dir, filename)
                # Load audio using TensorFlow
                audio_binary = tf.io.read_file(file_path)
                audio, sample_rate = tf.audio.decode_wav(audio_binary)
                waveform = tf.squeeze(audio, axis=-1)
                
                # Convert to spectrogram
                spectrogram = get_spectrogram(waveform)
                spectrogram = tf.image.resize(spectrogram, target_shape)

                data.append(spectrogram.numpy())
                labels.append(i)
                file_count += 1
                if file_count >= max_files_per_class:
                    break
    print('Data loading and preprocessing complete.')
    return np.array(data), np.array(labels)

# Define your folder structure
data_dir = 'data'
classes = [d for d in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, d))]
print('Classes:', classes)

# Load and preprocess the data with TensorFlow
data, labels = load_and_preprocess_data_tf(
    data_dir, classes, target_shape=target_shape, max_files_per_class=max_files_per_class)

# Convert labels to one-hot encoding
labels = to_categorical(labels, num_classes=len(classes))

# Create train_dataset and test_dataset
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

print('Loaded data shape:', X_train.shape, X_test.shape, y_train.shape, y_test.shape)

Classes: ['de', 'en', 'es', 'fr', 'it', 'pt', 'ru', 'tr']
Loading and preprocessing data with TensorFlow...
Processing class de (1/8)...


 24%|██▍       | 3133/12864 [01:06<03:26, 47.10it/s]

In [ ]:
# Show some random samples

def plot_spectrogram(spectrogram, ax):
    if len(spectrogram.shape) > 2:
        assert len(spectrogram.shape) == 3
        spectrogram = np.squeeze(spectrogram, axis=-1)
    # Convert the frequencies to log scale and transpose, so that the time is
    # represented on the x-axis (columns).
    # Add an epsilon to avoid taking a log of zero.
    log_spec = np.log(spectrogram.T + np.finfo(float).eps)
    height = log_spec.shape[0]
    width = log_spec.shape[1]
    X = np.linspace(0, np.size(spectrogram), num=width, dtype=int)
    Y = range(height)
    ax.pcolormesh(X, Y, log_spec)


def show_random_samples(data, labels, classes, num_samples=5):
    fig, axes = plt.subplots(1, num_samples, figsize=(20, 4))
    indices = np.random.choice(len(data), num_samples, replace=False)
    for i, ax in enumerate(axes):
        idx = indices[i]
        spectrogram = data[idx]
        label = classes[np.argmax(labels[idx])]
        plot_spectrogram(spectrogram, ax)
        ax.set_title(label)
    plt.show()


# Show random samples from the test set
show_random_samples(X_test, y_test, classes)

In [6]:
# Dataset cache and prefetch
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)
train_dataset = train_dataset.cache()
train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)

test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))
test_dataset = test_dataset.batch(batch_size)
test_dataset = test_dataset.cache()
test_dataset = test_dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
input_shape = X_train[0].shape

# Normalize the data
norm_layer = tf.keras.layers.Normalization()
norm_layer.adapt(X_train)

model = tf.keras.Sequential([
    Input(shape=input_shape),
    # Downsample the input.
    Resizing(32, 32),
    # Normalize.
    norm_layer,
    Conv2D(32, 3, activation='relu'),
    Conv2D(64, 3, activation='relu'),
    MaxPooling2D(),
    Dropout(0.25),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(classes), activation='softmax')
])

model.summary()

In [8]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [9]:
class EarlyStoppingByAccuracyDiff(tf.keras.callbacks.Callback):
    def __init__(self, threshold=0.2):
        super(EarlyStoppingByAccuracyDiff, self).__init__()
        self.threshold = threshold

    def on_epoch_end(self, epoch, logs=None):
        train_acc = logs.get('accuracy')
        val_acc = logs.get('val_accuracy')

        if train_acc is not None and val_acc is not None:
            if (train_acc - val_acc) > self.threshold:
                print(f"\nStopping training: train_acc={train_acc}, val_acc={val_acc}, diff={train_acc - val_acc}")
                self.model.stop_training = True

In [10]:
# Define the checkpoint callback to save the latest model
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='latest_model.keras',  # Use the .keras extension
    save_best_only=True,  # Save the best model only at the end of each epoch
    save_weights_only=False,  # Save the entire model
    monitor='val_accuracy',
    verbose=1
)

In [11]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [7]:
model = load_model('latest_model.keras')

In [ ]:
early_stop_callback = EarlyStoppingByAccuracyDiff(threshold=0.2)

# Train the model
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test), callbacks=[tensorboard_callback, checkpoint_callback, early_stop_callback])

In [ ]:
%tensorboard --logdir logs/fit

In [ ]:
test_accuracy=model.evaluate(X_test,y_test,verbose=0)
print(test_accuracy[1])

In [11]:
# Save the model
model.save('audio_classification_model.keras')

# Model prediction on audio

In [ ]:
# Load the saved model
model = load_model('audio_classification_model.keras')

target_shape = (128, 128)

# Function to preprocess and classify an audio file
def test_audio(file_path, model):
    # Load and preprocess the audio file
    audio_binary = tf.io.read_file(file_path)
    audio, sample_rate = tf.audio.decode_wav(audio_binary)
    waveform = tf.squeeze(audio, axis=-1)
    mel_spectrogram = get_spectrogram(waveform)
    mel_spectrogram = tf.image.resize(mel_spectrogram, target_shape)
    mel_spectrogram = mel_spectrogram.numpy()
    mel_spectrogram = mel_spectrogram[np.newaxis, ...]
    
    # Make predictions
    predictions = model.predict(mel_spectrogram)
    
    # Get the class probabilities
    class_probabilities = predictions[0]
    
    # Get the predicted class index
    predicted_class_index = np.argmax(class_probabilities)
    
    return class_probabilities, predicted_class_index

# Test an audio file
test_audio_file = "data/es/yqUcDCVhytk__U__S11---0082.030-0097.400.wav"
class_probabilities, predicted_class_index = test_audio(test_audio_file, model)

# Display results for all classes
for i, class_label in enumerate(classes):
    probability = class_probabilities[i]
    print(f'Class: {class_label}, Probability: {probability:.4f}')

# Calculate and display the predicted class and accuracy
predicted_class = classes[predicted_class_index]
accuracy = class_probabilities[predicted_class_index]
print(f'The audio is classified as: {predicted_class}')
print(f'Accuracy: {accuracy:.4f}')